# Manipulating tax-scales

In [ ]:
from openfisca_senegal import CountryTaxBenefitSystem as SenegalTaxBenefitSystem  # The Senegalese tax-benefits system
from openfisca_core.periods import period # Needed to specify the periods to modify the parameters

## Extract the tax-scale

In [ ]:
tax_benefit_system = SenegalTaxBenefitSystem()

In [ ]:
bareme_impot_progressif = tax_benefit_system.parameters.prelevements_obligatoires.impots_directs.bareme_impot_progressif

In [ ]:
bareme_impot_progressif

## Analyse the tax scale

In [ ]:
type(bareme_impot_progressif.brackets)

In [ ]:
len(bareme_impot_progressif.brackets)

It is a list of brackets.

## Analyse a bracket

In [ ]:
bracket = bareme_impot_progressif.brackets[3]
bracket

## Create a bracket

In [ ]:
from openfisca_core.parameters import Bracket

In [ ]:
another_bracket = Bracket(
    data = {
        'rate': {
            "2015-01-01": .1,
            "2013-01-01": .05
            },
        'threshold': {
            "2013-01-01": 1000
            }
        }
    )
another_bracket

## Modify the tax-scale

### Remove or add a bracket

Since it is a list we can delete an element ...

In [ ]:
del bareme_impot_progressif.brackets[5]

In [ ]:
len(bareme_impot_progressif.brackets)

In [ ]:
bareme_impot_progressif.brackets

... or add a bracket

In [ ]:
bareme_impot_progressif.brackets.insert(1, another_bracket)

In [ ]:
bareme_impot_progressif

### Modify a bracket of the tax-scale 

In [ ]:
bracket

In [ ]:
bareme_impot_progressif.brackets

In [ ]:
bracket.rate.update(start = period("2014-01-01"), stop = period("2015-12-31"), value = .33)
bracket

In [ ]:
bareme_impot_progressif

In [ ]:
bracket.threshold.update(start = period("2014-01-01"), stop = period("2015-12-31"), value = 5000000)

In [ ]:
bracket

In [ ]:
bareme_impot_progressif

# Microsimulation with artificial survey data 

In [ ]:
import matplotlib.pyplot as plt  # For graphics
%matplotlib inline

import numpy as np  # linear algebra and math
import pandas as pd  # data frames

from openfisca_core.model_api import *
from openfisca_senegal import CountryTaxBenefitSystem as SenegalTaxBenefitSystem  # The Senegalese tax-benefits system

from openfisca_senegal.survey_scenarios import SenegalSurveyScenario

In [ ]:
%config Completer.use_jedi = False

## Building the artificial data

Sénégal is composed by almost 15 millions people dispatched in around 1.6 million household

In [ ]:
household_weight = 100
size = int(1.6e6 / household_weight)
print(f"Size of the sample: {size}")
np.random.seed(seed = 42)

We assume that 2/3 of the household heads are married and that only married households do have children. The mean number of children per household is 5 and is normally distributed

In [ ]:
est_marie = np.random.binomial(1, .66, size = size)
est_celibataire = np.logical_not(est_marie)
nombre_enfants = np.maximum(
    est_marie * np.round(np.random.normal(5, scale = 3, size = size)),
    0,
    )

We assume that 80% of the population are wage earners.
We choose a mean wage of 5,000,000 CFA with a log normal ditribution.
Since 
$$ \text{mean wage}  = e^{\mu + \frac{\sigma ^ 2}{2}} $$ 
and
$$ \text{median wage} = e^\mu $$
we can compute the distribution according to the following expressions.

In [ ]:
mean_wage = 5e6
median_wage = .75 * mean_wage
est_salarie = np.random.binomial(1, .8, size = size)
mu = np.log(median_wage)
sigma = np.sqrt(2 * np.log(mean_wage / median_wage))
salaire = (
    est_salarie * 
    np.random.lognormal(mean = mu, sigma = sigma, size = int(size))
    )


We choose a mean pension of 2 500 000 CFA

In [ ]:
mean_pension = 2.5e6
median_pension = .9 * mean_pension

In [ ]:
mu = np.log(median_pension)
sigma = np.sqrt(2 * np.log(mean_pension / median_pension))
pension_retraite = (
    np.logical_not(est_salarie) *
    np.random.lognormal(mean = mu, sigma = sigma, size = int(size))
    )

In [ ]:
input_data_frame = pd.DataFrame({
    'est_marie': est_marie,
    'est_celibataire': est_celibataire,
    'nombre_enfants': nombre_enfants,
    'pension_retraite': pension_retraite,
    'salaire': salaire,
    'household_id': range(size),
    'household_role_index': 0,
    'household_weight': household_weight,
    'person_weight': household_weight,
    })

In [ ]:
input_data_frame.salaire.mean()

In [ ]:
input_data_frame.groupby(['est_marie', 'nombre_enfants'])['salaire'].max()

In [ ]:
input_data_frame.salaire.hist()

In [ ]:
input_data_frame.salaire.hist(bins = 100)

In [ ]:
input_data_frame.pension_retraite.hist(bins=100, range = [.0001, 1e7])

## Microsimulation 

### Scenario

As with test case, we can build a scenario with survey data

In [ ]:
data = dict(input_data_frame = input_data_frame)
scenario = SenegalSurveyScenario(
    data = data,
    period = 2017,
    )

In [ ]:
scenario.tax_benefit_systems["baseline"].variables.keys()

We can compute the value of any variable for the whole population and draw distributions

In [ ]:
scenario.simulations["baseline"].calculate('impot_revenus', period = 2017)

In [ ]:
df = pd.DataFrame(
    {'impot_revenus': scenario.simulations["baseline"].calculate('impot_revenus', period = 2017)}
)

In [ ]:
df.hist(bins = 100, range = [.00001, 2.5e7])

### Special scenario methods allow access to aggregates and pivot tables

In [ ]:
scenario.weight_variable_by_entity = {'person': 'person_weight', 'household': 'household_weight'}

In [ ]:
scenario.compute_aggregate('impot_revenus', period = 2017) / 1e9

In [ ]:
df = scenario.compute_pivot_table(
    aggfunc = 'sum', 
    values = ['impot_revenus'], 
    columns = ['nombre_enfants'],
    period = 2017,
    )
df

In [ ]:
df.transpose().reset_index()

In [ ]:
scenario.compute_pivot_table(
    aggfunc = 'sum', 
    values = ['impot_revenus'], 
    columns = ['nombre_enfants'],
    period = 2017,
    ).transpose().reset_index().plot(x = 'nombre_enfants', kind = 'bar')

## Evaluate the financial impact of a reform

### Parametric reform

Write a parametric reform that increases the top marginal tax rates and evaluate how much revenue can be collected

In [ ]:
year = 2017
def modify_parameters(parameters):
    parameters.prelevements_obligatoires.impots_directs.bareme_impot_progressif[5].rate.update(period = period(year), value = .5)
    return parameters
    
class tax_the_rich(Reform):
    name = "Tax last bracket at 50%"

    def apply(self):
        self.modify_parameters(modifier_function = modify_parameters)

In [ ]:
senegal_tax_benefit_system = SenegalTaxBenefitSystem()
tax_the_rich_tax_benefit_system = tax_the_rich(senegal_tax_benefit_system)

In [ ]:
scenario = SenegalSurveyScenario(
    data = data,
    tax_benefit_system = tax_the_rich_tax_benefit_system,
    baseline_tax_benefit_system = senegal_tax_benefit_system,
    period = 2017,
    )

In [ ]:
print('reform tax the rich: ', scenario.compute_aggregate('impot_revenus', period = 2017) / 1e9)
print('baseline: ', scenario.compute_aggregate('impot_revenus', period = 2017, use_baseline = True) / 1e9)

In [ ]:
scenario.compute_aggregate('impot_revenus', period = 2017, difference = True) / 1e9

### Structural reform

In [ ]:
from openfisca_senegal.entities import Person

def build_ultimate_reform_tax_benefit_system(threshold = 0, marginal_tax_rate = .4):
    year = 2017

    senegal_tax_benefit_system = SenegalTaxBenefitSystem()
    class impot_revenus(Variable):
        def formula(person, period):
            impot_avant_reduction_famille = person('impot_avant_reduction_famille', period)
            reduction_impots_pour_charge_famille = person('reduction_impots_pour_charge_famille', period)
            impot_apres_reduction_famille = impot_avant_reduction_famille - reduction_impots_pour_charge_famille
            impot_revenus = max_(0, impot_apres_reduction_famille)
            return impot_revenus * (impot_revenus > threshold)            

    def modify_parameters(parameters):
        parameters.prelevements_obligatoires.impots_directs.bareme_impot_progressif[5].rate.update(
            period = period(year), value = marginal_tax_rate)
        return parameters


    class ultimate_reform(Reform):
        name = f"Tax the rich and save the poor taxpayers (tax < {threshold})"

        def apply(self):
            self.update_variable(impot_revenus)
            self.modify_parameters(modifier_function = modify_parameters)

    return ultimate_reform(senegal_tax_benefit_system)

In [ ]:
reformed_tax_benefit_system = build_ultimate_reform_tax_benefit_system(
    threshold = 100000, 
    marginal_tax_rate = .5
    )

In [ ]:
scenario = SenegalSurveyScenario(
    data = data,
    tax_benefit_system = reformed_tax_benefit_system,
    baseline_tax_benefit_system = SenegalTaxBenefitSystem(),
    period = 2017
    )

In [ ]:
print('reform: ', scenario.compute_aggregate('impot_revenus', period = 2017) / 1e9)
print('baseline: ', scenario.compute_aggregate('impot_revenus', period = 2017, use_baseline = True) / 1e9)

In [ ]:
scenario = SenegalSurveyScenario(
    data = data,
    tax_benefit_system = reformed_tax_benefit_system,
    baseline_tax_benefit_system =  SenegalTaxBenefitSystem(),
    period = 2017
    )
print(scenario.compute_aggregate('impot_revenus', period = 2017) / 1e9)
print(scenario.compute_aggregate('impot_revenus', period = 2017, use_baseline = True) / 1e9)
cost = - (
    scenario.compute_aggregate('impot_revenus', period = 2017) -
    scenario.compute_aggregate('impot_revenus', period = 2017, use_baseline = True)
    ) / 1e9
print(cost)

### Budget neutral reforms

In [ ]:
def compute_reform_cost(threshold = 0, marginal_tax_rate = .4):
    reformed_tax_benefit_system = build_ultimate_reform_tax_benefit_system(
        threshold = threshold, 
        marginal_tax_rate = float(marginal_tax_rate)  
        # We need to convert to float here since fsolve use numpy array which are not accepted as 
        # legislation parameters
        )
    scenario = SenegalSurveyScenario(
        data = data,
        tax_benefit_system = reformed_tax_benefit_system,
        baseline_tax_benefit_system = SenegalTaxBenefitSystem(),
        period = 2017
        )
    cost = - (
        scenario.compute_aggregate('impot_revenus', period = 2017) - 
        scenario.compute_aggregate('impot_revenus', period = 2017, use_baseline = True)
        ) / 1e9
    return cost

In [ ]:
def compute_reform_cost_from_marginal_tax_rate(marginal_tax_rate = .4):
    return compute_reform_cost(threshold = 100000, marginal_tax_rate = marginal_tax_rate)

def compute_reform_cost_from_threshold(threshold = 0):
    return compute_reform_cost(threshold = threshold, marginal_tax_rate = .41)


In [ ]:
from scipy.optimize import fsolve
balancing_marginal_tax_rate = fsolve(compute_reform_cost_from_marginal_tax_rate, .40)
balancing_threshold = fsolve(compute_reform_cost_from_threshold, 1000)
print(f"balancing_marginal_tax_rate to finance a 100000 threshold: {float(balancing_marginal_tax_rate)}")
print(f"balancing_threshold financed by an increase from 40% to 41%: {float(balancing_threshold)}")

In [ ]:
print(compute_reform_cost_from_marginal_tax_rate(marginal_tax_rate = balancing_marginal_tax_rate))
print(compute_reform_cost_from_threshold(threshold = balancing_threshold))

In [ ]:
reformed_tax_benefit_system = build_ultimate_reform_tax_benefit_system(
    threshold = balancing_threshold, 
    marginal_tax_rate = .41
    )
scenario = SenegalSurveyScenario(
    data = data,
    tax_benefit_system = reformed_tax_benefit_system,
    baseline_tax_benefit_system = SenegalTaxBenefitSystem(),
    period = 2017
    )

In [ ]:
scenario.compute_pivot_table(
    aggfunc = 'sum', 
    values = ['impot_revenus'], 
    columns = ['nombre_enfants'],
    period = 2017,
    difference = True,
    ).stack().reset_index().plot(x = 'nombre_enfants', kind = 'bar')

In [ ]:
scenario.compute_pivot_table(
    aggfunc = 'mean', 
    values = ['impot_revenus'], 
    columns = ['nombre_enfants'],
    period = 2017,
    difference = True,
    ).stack().reset_index()

In [ ]:
scenario.compute_pivot_table(
    aggfunc = 'sum', 
    values = ['impot_revenus'], 
    columns = ['nombre_enfants'],
    period = 2017,
    difference = True,
    ).stack().sum() / 1e9
# The result is close to zero and almost equal to the one computed above